In [159]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [202]:
df = pd.read_csv(r"Book1.csv")
df = df.dropna()
df.head(2)

IN_TIME  CON_SIZE STATUS           OUT_TIME
0  12-11-22 01:29:54      40.0      E  07-02-23 01:23:13
1  22-11-22 13:40:39      40.0      E  02-02-23 19:33:43

In [161]:
df["IN_TIME"] = pd.to_datetime(df["IN_TIME"])
df["OUT_TIME"] = pd.to_datetime(df["OUT_TIME"])

In [162]:
featuresIn = ["In_year", "In_month", "In_day", "In_hour", "In_minute", "In_second", "In_weekday", "In_weekofyear", "In_quarter"]
featuresOut = ["Out_year", "Out_month", "Out_day", "Out_hour", "Out_minute", "Out_second", "Out_weekday", "Out_weekofyear", "Out_quarter"]

In [163]:
df[featuresIn] = df.apply(lambda row: pd.Series({"In_year":row.IN_TIME.year, "In_month":row.IN_TIME.month, "In_day":row.IN_TIME.day, "In_hour":row.IN_TIME.hour, "In_minute":row.IN_TIME.minute, "In_second":row.IN_TIME.second, "In_weekday":row.IN_TIME.weekday(), "In_weekofyear":row.IN_TIME.weekofyear, "In_quarter":row.IN_TIME.quarter }), axis=1)

In [164]:
df[featuresOut] = df.apply(lambda row: pd.Series({"Out_year":row.OUT_TIME.year, "Out_month":row.OUT_TIME.month, "Out_day":row.OUT_TIME.day, "Out_hour":row.OUT_TIME.hour, "Out_minute":row.OUT_TIME.minute, "Out_second":row.OUT_TIME.second, "Out_weekday":row.OUT_TIME.weekday(), "Out_weekofyear":row.OUT_TIME.weekofyear, "Out_quarter":row.OUT_TIME.quarter }), axis=1)

In [165]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()

encoded = oe.fit_transform(df[['STATUS']])
df['STATUS']=encoded
df.head(3)

IN_TIME  CON_SIZE  STATUS            OUT_TIME  In_year  \
0 2022-12-11 01:29:54      40.0     0.0 2023-07-02 01:23:13     2022   
1 2022-11-22 13:40:39      40.0     0.0 2023-02-02 19:33:43     2022   
2 2022-04-12 17:13:02      20.0     0.0 2023-10-02 05:09:09     2022   

   In_month  In_day  In_hour  In_minute  In_second  ...  In_quarter  Out_year  \
0        12      11        1         29         54  ...           4      2023   
1        11      22       13         40         39  ...           4      2023   
2         4      12       17         13          2  ...           2      2023   

   Out_month  Out_day  Out_hour  Out_minute  Out_second  Out_weekday  \
0          7        2         1          23          13            6   
1          2        2        19          33          43            3   
2         10        2         5           9           9            0   

   Out_weekofyear  Out_quarter  
0              26            3  
1               5            1  
2              40            4  

[3 rows x 22 columns]

In [166]:
X=df.drop(columns=['IN_TIME','OUT_TIME','Out_year', 'Out_month', 'Out_day', 'Out_hour', 'Out_weekday',
       'Out_weekofyear', 'Out_quarter', 'Out_minute',
       'Out_second'])

In [167]:
Y = df[['Out_year', 'Out_month', 'Out_day', 'Out_hour', 'Out_weekday',
       'Out_weekofyear', 'Out_quarter', 'Out_minute',
       'Out_second']]

In [168]:
Y.columns

Index(['Out_year', 'Out_month', 'Out_day', 'Out_hour', 'Out_weekday',
       'Out_weekofyear', 'Out_quarter', 'Out_minute', 'Out_second'],
      dtype='object')

In [169]:
X.columns

Index(['CON_SIZE', 'STATUS', 'In_year', 'In_month', 'In_day', 'In_hour',
       'In_minute', 'In_second', 'In_weekday', 'In_weekofyear', 'In_quarter'],
      dtype='object')

In [170]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [171]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
X_train.head()

CON_SIZE  STATUS  In_year  In_month  In_day  In_hour  In_minute  \
3161       40.0     1.0     2023         2      14       18         53   
10290      40.0     0.0     2023        10       3       12         36   
22636      40.0     1.0     2023         4      24       18          9   
4219       20.0     1.0     2023         8       2       15          0   
16188      40.0     1.0     2023         6       4       20         25   

       In_second  In_weekday  In_weekofyear  In_quarter  
3161           6           1              7           1  
10290         24           1             40           4  
22636         10           0             17           2  
4219          52           2             31           3  
16188          1           6             22           2

In [172]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=250,random_state=0)
regressor.fit(X_train,y_train)

RandomForestRegressor(n_estimators=250, random_state=0)

In [173]:
y_pred = regressor.predict(X_test)

In [174]:
r2_score(y_test, y_pred)

0.5520116668736512

In [196]:
import datetime as dt
con_size = 20
#E=0 L=1
datetime_str = '19/01/23 16:24:21'
date = dt.datetime.strptime(datetime_str, '%d/%m/%y %H:%M:%S')

In [197]:
#17-04-22 04:41:42
y_pred1 = regressior.predict([[con_size,1,
                               date.year,date.month,date.day,
                               date.hour,date.minute,date.second,
                               date.weekday(),date.isocalendar(). week,pd.Timestamp(date).quarter]])
y_pred1 =  [ int(x.round()) for x in y_pred1[0]]
y_pred1

[2023, 3, 2, 13, 3, 8, 1, 29, 32]

In [198]:
if y_pred1[1]>4:
    y_pred1[1]-=1
dt = dt.date(y_pred1[0],y_pred1[1],y_pred1[2])

In [199]:
dt

datetime.date(2023, 3, 2)

In [186]:
import pickle

In [187]:
pickle.dump(regressor,open('The Prediction model',mode='wb'))  #write binary